### Test the robustness of SCARP

In [1]:
import sys
sys.path.append('../Scarp/')
from downstream import *
import warnings
warnings.filterwarnings("ignore")
np.random.seed(1)

In [2]:
random_state = 1
m = 1.5
merge_thre = 3000
beta = 5000

In [3]:
data_type = 'blood2k'
Data_name = ['blood2K_filter30', 'blood2K_filter40',
             'blood2K_filter50', 'blood2K_filter60',
             'blood2K_filter70', 'blood2K_filter_var0.5',
             'blood2K_filter_var0.6', 'blood2K_filter_var0.7',
             'blood2K_filter_var0.8', 'blood2K_filter_var0.9']


In [4]:
# data_type = 'Sox10KD'
# Data_name = ['Sox10KD_filter10', 'Sox10KD_filter20',
#              'Sox10KD_filter30', 'Sox10KD_filter40',
#              'Sox10KD_filter50', 'Sox10KD_filter_var0.5',
#              'Sox10KD_filter_var0.6', 'Sox10KD_filter_var0.7',
#              'Sox10KD_filter_var0.8', 'Sox10KD_filter_var0.9']

In [5]:
louvain_ARI_score = pd.DataFrame(index=Data_name, columns=['score'])
louvain_NMI_score = pd.DataFrame(index=Data_name, columns=['score'])

running_time = np.zeros(len(Data_name))
Peaks_number = np.zeros(len(Data_name))
Kept_component = np.zeros(len(Data_name))

In [6]:
for d in range(len(Data_name)):
    data_name = Data_name[d]
    print('\n++++++++++++++++++++ %s ++++++++++++++++++++++++' % data_name)

    # ===========================================================================================
    data = sc.read_h5ad('./Processed data/' + data_name + '.h5ad')

    Cells = data.obs.index
    Cells_num, Peaks_num = data.X.shape
    N = Cells_num + Peaks_num
    labels = data.obs['celltype'].astype('category')
    cluster_num = np.unique(labels).shape[0]
    print('Number of Peaks:', Peaks_num)
    print('Number of Cells:', Cells_num)
    print('Number of labels: ', cluster_num)
    Peaks_number[d] = Peaks_num

    # ===========================================================================================
    t, diffusion_mat = SCARP(data=data,
                             m=m,
                             merge_thre=merge_thre,
                             beta=beta,
                             peak_loc=True)
    running_time[d] = t

    if Peaks_num > 50000:
        k = std_plot(data=diffusion_mat,
                     title=data_name,
                     max_k=100,
                     plot_std=False)
    else:
        k = std_plot(data=diffusion_mat,
                     title=data_name,
                     max_k=50,
                     plot_std=False)

    Kept_component[d] = k

    SCARP_score = compute_score(data_name=data_name,
                                diffusion_mat=diffusion_mat,
                                cluster_num=cluster_num,
                                labels=labels,
                                Cells=Cells,
                                kept_comp=k,
                                random_state=1
                                )

    louvain_ARI_score['score'][data_name] = SCARP_score['ARI']['louvain']
    louvain_NMI_score['score'][data_name] = SCARP_score['NMI']['louvain']


++++++++++++++++++++ blood2K_filter30 ++++++++++++++++++++++++
Number of Peaks: 105167
Number of Cells: 2034
Number of labels:  9

%%%%%%%%%%%%%%%%%%%%%%% Diffusion Started (with 23 cores parallel computing)%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%% Diffusion Finished %%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Running time: 251.4633982181549s

Matrix Splicing...........
Splicing time: 8.380032300949097s

++++++++++++++++++++ blood2K_filter40 ++++++++++++++++++++++++
Number of Peaks: 84480
Number of Cells: 2034
Number of labels:  9

%%%%%%%%%%%%%%%%%%%%%%% Diffusion Started (with 23 cores parallel computing)%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%% Diffusion Finished %%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Running time: 160.1779646873474s

Matrix Splicing...........
Splicing time: 7.116028308868408s

++++++++++++++++++++ blood2K_filter50 ++++++++++++++++++++++++
Number of Peaks: 70421
Number of Cells: 2034
Number of labels:  9

%%%%%%%%%%%%%%%%%%%%%%% Diffusion Started (with 23 cores par

## plot

In [7]:
louvain_ARI_score = louvain_ARI_score.astype('float')
louvain_NMI_score = louvain_NMI_score.astype('float')
running_time = running_time.astype('float')
Peaks_number = Peaks_number.astype('float')

louvain_ARI_score.to_csv('./Results/' + data_type +'_count_variance_filter_louvain_ARI_score.txt', sep='\t')
louvain_NMI_score.to_csv('./Results/' + data_type +'_count_variance_filter_louvain_NMI_score.txt', sep='\t')